## Session 1 - From Prompts to Partners: Building Your First AI Agent 

### Setup the environment

In [1]:
!wget -O requirements.txt -q https://raw.githubusercontent.com/Sage/talent-arena-2026/master/requirements.txt
!wget -O utils.py -q https://raw.githubusercontent.com/Sage/talent-arena-2026/master/utils.py
!python -m ensurepip --upgrade
%pip install --upgrade pip
%pip install -r requirements.txt

Looking in links: /var/folders/q9/50rk3g6d7y1251dhvdlf5jk40000gn/T/tmp6wowiiuq
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Implement your agent

In [2]:
from utils import chat
chat()

Hello from talent-arena-2026!
